In [1]:
import psycopg2
import os
import csv
import sys
sys.path.append('../')
from psycopg2 import sql
import numpy as np

from src.utils import connect_db, get_table_name
from upload.upload_PostgreSQL import upload_PostgreSQL
# from create.create_tables import get_table_info, get_column_names

In [3]:
import psycopg2
import os
import csv
import sys
sys.path.append('../')
from psycopg2 import sql
import numpy as np
import json
from src.utils import connect_db, get_table_name


def upload_PostgreSQL(table_name,
                      proto_name,
                       material,
                       geometry,
                       resolution,
                       thickness,
                       actual_X,
                       actual_Y,
                       actual_Z,
                       flatness,
                       inspectDate,
                       inspectTime,
                       comments):
    
    conn = connect_db()
    cursor = conn.cursor()
    print('Connection successful. \n')

    schema_name = 'public'
    table_exists_query = sql.SQL("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = %s AND table_name = %s);")
    cursor.execute(table_exists_query, [schema_name, table_name])
    table_exists = cursor.fetchone()[0]## this gives true/false
    
    query = f""" 
    INSERT INTO {table_name} 
    (proto_name, flatness, thickness, x_points, y_points, z_points, inspector, comment)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    # date_inspect, time_inspect,

    
    from datetime import datetime
    print(datetime.now().strftime("%Y-%m-%d"),datetime.now().strftime("%H%M%S"))
    data = (proto_name, flatness, thickness, actual_X.tolist(), actual_Y.tolist(), actual_Z.tolist(), #str(datetime.now().strftime("%Y-%m-%d")),str(datetime.now().strftime("%H%M%S")),
            inspector, comments)
    print(tuple(data))
    if table_exists:
        cursor.execute(query, tuple(data))
        conn.commit()
        print(f'Data is successfully uploaded to the {table_name}!')
    
    else:
        print(f'Table {table_name} does not exist in the database.')
        
    ## close connection
    cursor.close()
    conn.close()
    
    return None



inspector = 'cmu_person'
resolution = 'LD'
geometry = 'full'
material = 'cf'
proto_name='test2'
comments = 'none'
flatness = '0'
inspectDate, inspectTime = 0,0
sensor_Heights = np.loadtxt('sensor_heights.txt')
actual_X, actual_Y, actual_Z = sensor_Heights[0], sensor_Heights[1], sensor_Heights[2]
print(actual_X.tolist())
thickness = np.mean(actual_Z)
table_name = "proto_inspect"
print('Data Obtained')
upload_PostgreSQL(table_name,
                    proto_name,
                       material,
                       geometry,
                       resolution,
                       thickness,
                       actual_X,
                       actual_Y,
                       actual_Z,
                       flatness,
                       inspectDate,
                       inspectTime,
                       comments)


[133.83667, 149.79117, 178.76729, 210.39729, 220.83415, 217.92127, 219.63354, 211.09385, 178.42969, 147.72378, 133.0574, 99.69919, 71.7943, 61.75042, 61.88726, 62.04552, 65.48081, 101.53708, 116.79271, 160.3848, 178.3835, 162.80256, 116.40882, 103.87269, 144.41987]
Data Obtained
Get superuser password: hgcal
Database name: testdb3
Connection successful. 

2024-02-07 180654
('test2', '0', 1.5632304, [133.83667, 149.79117, 178.76729, 210.39729, 220.83415, 217.92127, 219.63354, 211.09385, 178.42969, 147.72378, 133.0574, 99.69919, 71.7943, 61.75042, 61.88726, 62.04552, 65.48081, 101.53708, 116.79271, 160.3848, 178.3835, 162.80256, 116.40882, 103.87269, 144.41987], [199.86229, 200.19172, 219.67347, 239.97654, 250.29375, 285.83019, 327.29166, 340.95988, 359.20982, 377.38311, 376.43572, 356.57642, 339.76424, 325.40876, 289.5707, 248.47532, 241.82763, 221.5252, 254.48036, 254.65085, 287.3542, 322.13092, 321.94996, 290.30187, 291.29881], [1.47164, 1.43834, 1.4504, 1.45472, 1.44709, 1.51935, 1.5

In [2]:
fname = '../upload/sensor_Heights.txt'
table_name = 'proto_assembly'
numeric_data = np.loadtxt(fname)

In [3]:
## the columns accept numeric data are 
## position, region, ass_tray_id, comp_tray_id, put_id

numeric_data = numeric_data.reshape(-1, 5)
numeric_data.shape

(15, 5)

In [4]:
numeric_data

array([[133.83667, 149.79117, 178.76729, 210.39729, 220.83415],
       [217.92127, 219.63354, 211.09385, 178.42969, 147.72378],
       [133.0574 ,  99.69919,  71.7943 ,  61.75042,  61.88726],
       [ 62.04552,  65.48081, 101.53708, 116.79271, 160.3848 ],
       [178.3835 , 162.80256, 116.40882, 103.87269, 144.41987],
       [199.86229, 200.19172, 219.67347, 239.97654, 250.29375],
       [285.83019, 327.29166, 340.95988, 359.20982, 377.38311],
       [376.43572, 356.57642, 339.76424, 325.40876, 289.5707 ],
       [248.47532, 241.82763, 221.5252 , 254.48036, 254.65085],
       [287.3542 , 322.13092, 321.94996, 290.30187, 291.29881],
       [  1.47164,   1.43834,   1.4504 ,   1.45472,   1.44709],
       [  1.51935,   1.58971,   1.59909,   1.58758,   1.5761 ],
       [  1.56672,   1.56058,   1.56788,   1.58507,   1.62991],
       [  1.56062,   1.54363,   1.5323 ,   1.63208,   1.59806],
       [  1.60768,   1.63661,   1.63772,   1.66301,   1.62487]])

In [5]:
list(numeric_data[0])

[133.83667, 149.79117, 178.76729, 210.39729, 220.83415]

In [6]:
## At this point, let's say each column in numeric data represents the data for each column in the table
position = numeric_data[:, 0]
region = numeric_data[:, 1]
ass_tray_id = numeric_data[:, 2]
comp_tray_id = numeric_data[:, 3]
put_id = numeric_data[:, 4]

In [7]:
## other columns with text entries are
## proto_name, bp_name, sen_name, tape_batch, epoxy_batch, operator

proto_name = 'proto_name'
bp_name = 'bp_name'
sen_name = 'sen_name'
tape_batch = 'tape_batch'
epoxy_batch = 'epoxy_batch'
operator = 'cmu_person'

## DATE/ TIME entires
## date_inspect, time_inspect
date_inspect = '01-22-2024'
time_inspect = '10-25'

In [8]:
position.shape

(15,)

In [9]:
## in proto_assembly, there are 13 columns to be inserted data. 
## whereas the default number of columns in the upload_PostgreSQL is 11.
## So, here we will add two more columns (as in arguments in the function)

In [29]:
# %load upload_PostgreSQL.py
import psycopg2
import os
import csv
import sys
sys.path.append('../')
from psycopg2 import sql
import numpy as np
from src.utils import connect_db, get_table_name

def upload_PostgreSQL(table_name,
                        common_info1,
                        common_info2,
                        common_info3,
                        common_info4,
                        common_info5,
                        common_info6,
                        common_info7,
                        common_info8,
                        numeric_col1,
                        numeric_col2,
                        numeric_col3,
                        numeric_col4,
                        numeric_col5,
                        schema_name = 'public'):
    
    ## connect to database
    conn = connect_db()
    cursor = conn.cursor()
    print('Connection successful. \n')
        
    table_exists_query = sql.SQL("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = %s AND table_name = %s);")
    cursor.execute(table_exists_query, [schema_name, table_name])
    table_exists = cursor.fetchone()[0]## this gives true/false
    
#     data = (common_info1, common_info2, common_info3, common_info4, common_info5, common_info6,
#             common_info7, common_info8, numeric_col1, numeric_col2, numeric_col3, numeric_col4, numeric_col5)
    
    if table_exists:
#         query = f""" 
#                 INSERT INTO {table_name} 
#                 ({common_info1, common_info2, common_info3, common_info4, common_info5, 
#                   common_info6, common_info7, common_info8, numeric_col1, numeric_col2, 
#                   numeric_col3, numeric_col4, numeric_col5})
#                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#                 """
        columns = f'{common_info1}, {common_info2}, {common_info3}, {common_info4}, {common_info5}, {common_info6}, {common_info7}, {common_info8}, {numeric_col1}, {numeric_col2}, {numeric_col3}, {numeric_col4}, {numeric_col5}'
        
        query = sql.SQL("INSERT INTO {} ({}) VALUES({})").format(
                sql.Identifier(table_name),
                sql.Identifier(columns),
                sql.SQL('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s')
                )
        
        
        ## set up data structure
        common_info = [common_info1, common_info2, common_info3, common_info4, common_info5, common_info6, 
                       common_info7, common_info8]

        for i in range(numeric_col1.shape[0]):
            data = [numeric_col1[i], numeric_col2[i], numeric_col3[i], numeric_col4[i], numeric_col5[i]]
            data = common_info + data
            print(len(data), data)
            cursor.execute(query, tuple(data))
            
        conn.commit()
        print(f'Data is successfully uploaded to the {table_name}!')
    
    else:
        print(f'Table {table_name} does not exist in the database.')
        
    ## close connection
    cursor.close()
    conn.close()
    
    return None


In [34]:
params = {'table_name': table_name, 
          'common_info1': proto_name, 
          'common_info2': bp_name,
          'common_info3': sen_name,
          'common_info4': tape_batch,
          'common_info5': epoxy_batch,
          'common_info6': date_inspect,
          'common_info7': time_inspect,
          'common_info8': operator,
          'numeric_col1': position,
          'numeric_col2': region,
          'numeric_col3': ass_tray_id,
          'numeric_col4': comp_tray_id,
          'numeric_col5': put_id}


In [35]:
upload_PostgreSQL(**params)

Set superuser password: /nhgcal
Connection successful. 

13 ['proto_name', 'bp_name', 'sen_name', 'tape_batch', 'epoxy_batch', '01-22-2024', '10-25', 'cmu_person', 133.83667, 149.79117, 178.76729, 210.39729, 220.83415]


UndefinedColumn: column "proto_name, bp_name, sen_name, tape_batch, epoxy_batch, 01-22-2" of relation "proto_assembly" does not exist
LINE 1: INSERT INTO "proto_assembly" ("proto_name, bp_name, sen_name...
                                      ^
